# NewsAPI  - Getting articles
This API can get news articles from different sources and different topics.  

Documentation for Python API can be found at:  
https://github.com/mattlisiv/newsapi-python



In [1]:
pip install newsapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install newsapi-python

In [3]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Get your free API key from https://newsapi.org/, just need to sign up for an account
key = "56ba1070c68b4258a0bb25638215c516"

In [6]:
from newsapi import NewsApiClient
import json
from kafka import KafkaProducer

# Initialize API endpoint
newsapi = NewsApiClient(api_key=key)

# Define the list of media sources
sources = 'bbc-news,cnn,fox-news,nbc-news,the-guardian-uk,the-new-york-times,the-washington-post,usa-today,independent,daily-mail'

# /v2/everything - note there are other parameters you can set
all_articles = newsapi.get_everything(q='france',
                                      sources=sources,
                                      language='en')

# Print the titles of the articles
for article in all_articles['articles']:
    print(article['title'])



UK and EU to sign new deal on small boat crossings
France makes abortion constitutional right
Ireland beat Wales to make it three wins in three
Small boat smuggling gang 'dismantled' by police
Juventus' Pogba banned for four years for doping
Huawei's offices in France raided by financial prosecutors
Fly-half Costelow returns for Wales trip to Dublin
Nicolas Sarkozy gets six months for illegal campaign funding
French warships intercept drones from Yemen
Pioneering pet photographs revealed by museum
The Papers: 'King has cancer' and 'women dominate Grammys'
Europe farmers protests: EU scraps plans to halve pesticide use
How Russia has rebranded Wagner in Africa
'Unfair' to say Borthwick only data driven - Sinfield
Nato chief says Donald Trump comments 'undermine all of our security'
Scotland v England - where will the game be won?
Where and when is Eurovision 2024 taking place?
Dupont try helps France win sevens bronze
Six Nations: France v Italy - radio & text
The red card that banned D